In [20]:
import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels

url = 'http://berkeleyearth.lbl.gov/auto/Global/Land_and_Ocean_complete.txt'

# Get all Text
text = [l[:-1].decode("utf-8") for l in urllib.request.urlopen(url).readlines()]

# Pull out the info and display
info = {i: l[1:] for i,l in enumerate(text) if l[0] == '%'}
# for i in info.values():
#     print(i)
    
# Get the last info (separator between air/water temperatures)
separator = max(info.keys())


air_data = [l.split() for i,l in enumerate(text) if l[0] != '%' and len(l)>1 and i<separator]
water_data = [l.split() for i,l in enumerate(text) if l[0] != '%' and len(l)>1 and i>=separator]

columns = ['year', 'month', 'month_avg', 'ma_err', 'year_avg', 'ya_err', '5_year', '5_err',
           '10_year', '10_err', '20_year', '20_err']
air_df = pd.DataFrame(air_data, columns=columns)
water_df = pd.DataFrame(water_data, columns=columns)

dec_cols = ['month_avg', 'ma_err', 'year_avg', 'ya_err', '5_year', '5_err', '10_year', '10_err',
            '20_year', '20_err']
for df in (air_df, water_df):
    df[['year','month']] = df[['year','month']].astype('int')
    df[dec_cols] = df[dec_cols].astype(float)
    df['yr_mnth'] = df['year'] + df['month']/12
    df.set_index('yr_mnth', inplace=True)
    
air_df = air_df.rename(columns={'month_avg':'AIR_MONTH_AVG_TEMP'})
water_df = water_df.rename(columns={'month_avg':'WATER_MONTH_AVG_TEMP'})

df = air_df.merge(water_df, on=['year','month'])[['year','month','AIR_MONTH_AVG_TEMP','WATER_MONTH_AVG_TEMP']]
df = df.rename(columns={'year':'YEAR', 'month':'MONTH'})
display(df)
df.to_pickle("temperatures.pkl")
print("saved")

,YEAR,MONTH,AIR_MONTH_AVG_TEMP,WATER_MONTH_AVG_TEMP
0,1850,1,-0.725,-0.691
1,1850,2,-0.179,-0.177
2,1850,3,-0.348,-0.380
3,1850,4,-0.577,-0.534
4,1850,5,-0.600,-0.561
...,...,...,...,...
2048,2020,9,1.017,0.862
2049,2020,10,0.926,0.775
2050,2020,11,1.064,0.888
2051,2020,12,0.855,0.706


saved
